In [45]:
import pandas as pd
import hazm as hz
import pickle
import nltk

In [46]:
dataset = pd.read_csv('./datasets/Snappfood - Sentiment Analysis.csv', delimiter="\t")
dataset

,comment,label,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0
4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0
...,...,...,...
69995,سلام من به فاکتور غذاهایی که سفارش میدم احتیاج...,SAD,1
69996,سایز پیتزا نسبت به سفارشاتی که قبلا گذشتم کم ش...,SAD,1
69997,من قارچ اضافه رو اضافه کرده بودم بودم اما اگر ...,HAPPY,0
69998,همرو بعد ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو...,SAD,1


In [47]:
dataset["label"].value_counts()

SAD      35000
HAPPY    35000
Name: label, dtype: int64

In [48]:
# there is two class of SAD and HAPPY in dataset, take 20% of data with the same class ditribution
dataset = pd.concat([dataset[dataset['label'] == 'HAPPY'].sample(frac=0.2, random_state=97), dataset[dataset['label'] == 'SAD'].sample(frac=0.2, random_state=97)]).reset_index()
dataset["label"].value_counts()

HAPPY    7000
SAD      7000
Name: label, dtype: int64

In [49]:
# apply the Normalization and Tokenization to Dataset

import hazm as hz
comments = dataset["comment"].to_list()
transformed = []
for comment in comments:
    comment = hz.Normalizer().normalize(comment)
    comment = hz.word_tokenize(comment)
    transformed.append(comment)
dataset["comment"] = transformed

In [57]:
file = open('./datasets/preprocessed.pkl', 'wb')
with open('./datasets/preprocessed.pkl', 'wb') as f:
    pickle.dump(dataset, f)